In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your zip file

Saving train.rar to train.rar


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

# Install required packages for RAR extraction
!pip install patool pyunpack rarfile
!apt-get install unrar  # For Colab Linux environment

# Now import after installation
import patoolib
from pyunpack import Archive
import rarfile

# Function to safely extract archives
def extract_archive(archive_path, extract_to):
    try:
        if archive_path.endswith('.rar'):
            with rarfile.RarFile(archive_path) as rf:
                rf.extractall(extract_to)
        else:
            Archive(archive_path).extractall(extract_to)
        print(f"Successfully extracted {archive_path} to {extract_to}")
    except Exception as e:
        print(f"Error extracting {archive_path}: {str(e)}")
        # Try alternative method
        try:
            patoolib.extract_archive(archive_path, outdir=extract_to)
        except Exception as e2:
            print(f"Alternative extraction also failed: {str(e2)}")
            return False
    return True

# Extract the RAR files
extract_archive("/content/train.rar", "/content/train")
extract_archive("/content/test.rar", "/content/test")

# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=10,
                                  zoom_range=0.1,
                                  horizontal_flip=True)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Verify extraction worked by checking directory structure
if os.path.exists("/content/train") and os.path.isdir("/content/train"):
    # Preprocess all training images
    train_generator = train_data_gen.flow_from_directory(
            '/content/train',
            target_size=(48, 48),
            batch_size=64,
            color_mode="grayscale",
            class_mode='categorical')
else:
    raise FileNotFoundError("Training directory not found after extraction")

if os.path.exists("/content/test") and os.path.isdir("/content/test"):
    # Preprocess all validation images
    validation_generator = validation_data_gen.flow_from_directory(
            '/content/test',
            target_size=(48, 48),
            batch_size=64,
            color_mode="grayscale",
            class_mode='categorical')
else:
    raise FileNotFoundError("Validation directory not found after extraction")

# [Rest of your model code remains the same...]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
Successfully extracted /content/train.rar to /content/train
Successfully extracted /content/test.rar to /content/test
Found 28709 images belonging to 1 classes.
Found 7178 images belonging to 1 classes.


In [ ]:
# Build CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes, activation='softmax'))  # Output layer

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
# Set callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[early_stop]
)


In [ ]:
# Save the trained model
model.save('model.h5')
print("Model saved successfully!")


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('model.h5')


In [ ]:
# Step 1: Install Gradio
!pip install gradio --quiet

# Step 2: Upload model.h5 manually
from google.colab import files
uploaded = files.upload()

# Step 3: Import Libraries
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Step 4: Load your trained model
model = load_model('model.h5')

# Step 5: Define emotion classes
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
# <-- Update this list if your model has different emotions

# Step 6: Define prediction function
def predict(img):
    if img is None:
        return "No image uploaded"

    # Convert to numpy array
    img = np.array(img)

    # Resize image first
    img = image.smart_resize(img, (48, 48))

    # If RGB, convert to grayscale
    if img.ndim == 3 and img.shape[2] == 3:
        img = np.mean(img, axis=-1, keepdims=True)  # maintain (48,48,1)

    # Normalize
    img = img / 255.0

    # Reshape for model
    img = img.reshape(1, 48, 48, 1)

    # Predict
    prediction = model.predict(img)
    class_idx = np.argmax(prediction)
    emotion = class_names[class_idx]
    confidence = float(np.max(prediction)) * 100  # in percentage

    # Return result nicely
    return f"Emotion: {emotion} ({confidence:.2f}%)"

# Step 7: Create and launch Gradio app
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="numpy"),
    outputs=gr.Textbox(),  # <-- change output to simple text
    title="Facial Emotion Detector",
    description="Upload a face image and predict the person's emotion (Happy, Angry, Sad, etc.)"
)

interface.launch(share=True, debug=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.4 MB/s eta 0:00:00


Saving model.h5 to model.h5


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c9ec36644a66c96e06.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/keras/src/ops/nn.py:907: UserWarning: You are using a softmax over axis -1 of a tensor of shape (1, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
